<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/Evaluation/Exact_Match_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize Libraries


In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 15.4 MB/s eta 0:00:00


In [3]:
import torch
from torchmetrics.classification import ExactMatch
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

# Reading the Dataset

In [4]:
path = "/content/drive/MyDrive/Bachelor\'s project/bug_evaluation_dataset.csv"
df = pd.read_csv(path)
print(df.shape)
df.head()

(25793, 5)


,original_code,modified_code,changed_line,number_of_line,mutation_type
0,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"for j in range(y-1, r):",14,statement
1,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,if k<1:,8,decision
2,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,if k<1:,8,decision
3,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"for j in range(l&1, r):",14,operation
4,n = int(input())\nw = [2] * n\nans0 = n\nans =...,n = int(input())\nw = [2] * n\nans0 = n\nans =...,"l, r, k = list(z(int, input().split()))",7,statement


# EM Evaluation

In [9]:
# Tokenize by splitting on spaces, special characters can be handled more sophisticatedly
def tokenize_code(code):
    return code.split()

# Convert token sequences to labels (integers)
def tokenize_and_encode(code1, code2):
    tokens1 = tokenize_code(code1)
    tokens2 = tokenize_code(code2)

    # Combine tokens to get a consistent vocabulary
    all_tokens = tokens1 + tokens2

    # Use LabelEncoder to convert tokens to integer labels
    label_encoder = LabelEncoder()
    label_encoder.fit(all_tokens)

    # Convert both code snippets to sequences of labels
    token_labels1 = label_encoder.transform(tokens1)
    token_labels2 = label_encoder.transform(tokens2)

    return torch.tensor([token_labels1]), torch.tensor([token_labels2])

In [11]:
# Sample code snippets
code1 = """
n = int(input())
w = [2] * n
ans = []
q = int(input())
for i in range(q):
    l, r, k = list(map(int, input().split()))
    if k == 1:
        for j in range(l-1, r):
            if w[j] == 2:
                ans0 = ans0 - 1
                w[j] = 1
    else:
        for j in range(l-1, r):
            if w[j] == 1:
                ans0 = ans0 + 1
                w[j] = 2
    ans.append(ans0)
for i in range(q):
    print(ans[i])
"""

code2 = """
n = int(input())
w = [2] * n
ans = []
q = int(input())
for i in range(q):
    l, r, k = list(map(int, input().split()))
    if k == 1:
        for j in range(l-1, r):
            if w[j] == 2:
                ans0 = ans0 - 1
                w[j] = 1
    else:
        for j in range(l-1, r):
            if w[j] == 1:
                ans0 = ans0 + 1
                w[j] = 2
    ans.append(ans0)
for i in range(q):
    print(ans[i])
"""

# Get tokenized and encoded versions of the two code snippets
preds, target = tokenize_and_encode(code1, code2)

# Flatten the tensors
flat_preds = preds.view(-1)
flat_target = target.view(-1)

# Calculate exact match by checking if both tensors are identical
exact_match_score = 1 if torch.equal(flat_preds, flat_target) else 0

print(f"Exact Match Accuracy: {exact_match_score}")

Exact Match Accuracy: 1
